Convert HDF5 merged-tract file to Parquet.

Requires `fastparquet` or `pyarrow`

In [1]:
import os

import pandas as pd
import fastparquet as fp

In [2]:
object_table_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/object_catalog/'

In [3]:
prefix = 'trim_'
tract = 4848

In [4]:
object_table_basename = '{}merged_tract_{}'.format(prefix, tract)

In [29]:
hdf5_filepath = os.path.join(object_table_dir, object_table_basename+'.hdf5')
parquet_filepath = os.path.join(object_table_dir, object_table_basename+'.parquet')

In [30]:
nx, ny = 8, 8
patches = ['{:1d}{:1d}'.format(x, y) for x in range(nx) for y in range(ny)]

### 2018-08-21 MWV
Trying to append in Parquet is bringing up the same problem that @yymao has been asking about.  If the number of columns was fixed, that would greatly simplify reading by the Generic Catalog Reader.  But because each filter coadd adds an entire set of columns, when processing tract+patches with missing filters, I've just left them out.

This might be the push that I need to make the change.

In [35]:
verbose=True

print(parquet_filepath)

append = False  # Create new file the first time around.
for patch in patches:
    key = 'coadd_{}_{}'.format(tract, patch)
    try:
        df = pd.read_hdf(hdf5_filepath, key=key)
    except KeyError as ke:
        if verbose:
            print(ke)
        continue

    fp.write(parquet_filepath, df, append=append)
    append = True  # Once it exists, append subsequent writes

/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/object_catalog/trim_merged_tract_4848.parquet


KeyError: 'i_base_PsfFlux_fluxSigma'

In [22]:
help(os.rmdir
    )

Help on built-in function rmdir in module posix:

rmdir(path, *, dir_fd=None)
    Remove a directory.
    
    If dir_fd is not None, it should be a file descriptor open to a directory,
      and path should be relative; path will then be relative to that directory.
    dir_fd may not be implemented on your platform.
      If it is unavailable, using it will raise a NotImplementedError.



In [14]:
pf = fp.ParquetFile(parquet_filepath)

In [15]:
df = pf.to_pandas()

In [16]:
len(df)

708

In [17]:
help(fp.write)

Help on function write in module fastparquet.writer:

write(filename, data, row_group_offsets=50000000, compression=None, file_scheme='simple', open_with=<function default_open at 0x2ae847596378>, mkdirs=<function default_mkdirs at 0x2ae8475962f0>, has_nulls=True, write_index=None, partition_on=[], fixed_text=None, append=False, object_encoding='infer', times='int64')
    Write Pandas DataFrame to filename as Parquet Format
    
    Parameters
    ----------
    filename: string
        Parquet collection to write to, either a single file (if file_scheme
        is simple) or a directory containing the metadata and data-files.
    data: pandas dataframe
        The table to write
    row_group_offsets: int or list of ints
        If int, row-groups will be approximately this many rows, rounded down
        to make row groups about the same size; if a list, the explicit index
        values to start new row groups.
    compression: str, dict
        compression to apply to each column, 